# Q1 Warehouse

In [ ]:
import random
import heapq
from collections import deque

In [ ]:
class Warehouse:
    def __init__(self, seed):
        random.seed(seed)
        self.N = random.randint(5, 10)
        self.M = random.randint(5, 10)
        self.P = random.randint(2, 6)
        self.O = random.randint(1, 10)
        self.start = (0, 0)
        self.packages = []
        self.dropoffs = []
        self.obstacles = set()
        
        occupied = {self.start}
        
        # Generate packages and dropoffs
        for _ in range(self.P):
            while True:
                p = (random.randint(0, self.N-1), random.randint(0, self.M-1))
                if p not in occupied:
                    break
            occupied.add(p)
            
            while True:
                d = (random.randint(0, self.N-1), random.randint(0, self.M-1))
                if d not in occupied:
                    break
            occupied.add(d)
            
            self.packages.append(p)
            self.dropoffs.append(d)
        
        # Generate obstacles
        for _ in range(self.O):
            while True:
                o = (random.randint(0, self.N-1), random.randint(0, self.M-1))
                if o not in occupied:
                    self.obstacles.add(o)
                    occupied.add(o)
                    break

    def display_grid(self):
        grid = [['.' for _ in range(self.M)] for _ in range(self.N)]
        grid[self.start[0]][self.start[1]] = 'S'
        
        for x, y in self.packages:
            grid[x][y] = 'P'
        for x, y in self.dropoffs:
            grid[x][y] = 'D'
        for x, y in self.obstacles:
            grid[x][y] = 'O'
        
        print("Initial Warehouse Configuration:")
        for row in grid:
            print(' '.join(row))
        print()

In [ ]:
class Agent:
    def __init__(self, warehouse):
        self.warehouse = warehouse
        self.current_pos = warehouse.start
        self.total_cost = 0
        self.total_reward = 0
        self.penalties = 0
        self.path_taken = [warehouse.start]

    def get_neighbors(self, pos):
        x, y = pos
        return [(x+dx, y+dy) for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)] 
                if 0 <= x+dx < self.warehouse.N and 0 <= y+dy < self.warehouse.M]

    def bfs(self, start, goal):
        queue = deque([[start]])
        visited = set([start])
        
        while queue:
            path = queue.popleft()
            current = path[-1]
            if current == goal:
                return path
            for neighbor in self.get_neighbors(current):
                if neighbor not in visited and neighbor not in self.warehouse.obstacles:
                    visited.add(neighbor)
                    queue.append(path + [neighbor])
        return None

    def dfs(self, start, goal):
        stack = [[start]]
        visited = set([start])
        
        while stack:
            path = stack.pop()
            current = path[-1]
            if current == goal:
                return path
            for neighbor in reversed(self.get_neighbors(current)):
                if neighbor not in visited and neighbor not in self.warehouse.obstacles:
                    visited.add(neighbor)
                    stack.append(path + [neighbor])
        return None

    def ucs(self, start, goal):
        heap = []
        heapq.heappush(heap, (0, [start]))
        visited = {start: 0}
        
        while heap:
            cost, path = heapq.heappop(heap)
            current = path[-1]
            
            if current == goal:
                return path
                
            for neighbor in self.get_neighbors(current):
                if neighbor in self.warehouse.obstacles:
                    continue
                    
                new_cost = cost + 1  # Movement cost = 1 per step
                if neighbor not in visited or new_cost < visited[neighbor]:
                    visited[neighbor] = new_cost
                    heapq.heappush(heap, (new_cost, path + [neighbor]))
        return None

    def find_path(self, start, goal, algorithm):
        if algorithm == 'bfs':
            return self.bfs(start, goal)
        elif algorithm == 'dfs':
            return self.dfs(start, goal)
        elif algorithm == 'ucs':
            return self.ucs(start, goal)
        raise ValueError("Invalid algorithm")

    def deliver_packages(self, algorithm='ucs'):
        for i in range(len(self.warehouse.packages)):
            # Move to package
            pkg_path = self.find_path(self.current_pos, 
                                    self.warehouse.packages[i], 
                                    algorithm)
            if not pkg_path:
                print(f"Package {i+1} unreachable!")
                continue
                
            self.path_taken += pkg_path[1:]
            self.total_cost += len(pkg_path) - 1
            self.current_pos = self.warehouse.packages[i]
            
            # Move to dropoff
            drop_path = self.find_path(self.current_pos, 
                                     self.warehouse.dropoffs[i], 
                                     algorithm)
            if not drop_path:
                print(f"Dropoff {i+1} unreachable!")
                continue
                
            self.path_taken += drop_path[1:]
            self.total_cost += len(drop_path) - 1
            self.current_pos = self.warehouse.dropoffs[i]
            self.total_reward += 10
            
        final_score = self.total_reward - self.total_cost - (self.penalties * 5)
        return {
            'algorithm': algorithm,
            'path': self.path_taken,
            'total_cost': self.total_cost,
            'total_reward': self.total_reward,
            'final_score': final_score
        }

In [ ]:
seed = 42
warehouse = Warehouse(seed)
warehouse.display_grid()
for algorithm in ["bfs","dfs","ucs"]:
    print(f"\nResults using {algorithm.upper()}:")

    agent = Agent(warehouse)
    results = agent.deliver_packages(algorithm)

    print(f"Path: {' -> '.join([ str(i) for i in results['path']])}")
    print(f"Total Movement Cost: {results['total_cost']}")
    print(f"Delivery Rewards: {results['total_reward']}")
    print(f"Final Score: {results['final_score']}")


# Q2 Different Cities 

In [ ]:
from IPython.display import display, HTML
# import matplotlib.pyplot as plt
import warnings,sys, time, random, heapq
warnings.filterwarnings('ignore')
from tqdm import tqdm
from pprint import pprint
from shapely.geometry import shape, Point
import geopandas as gpd
import networkx as nx

random.seed(42)

In [ ]:
city_gdf = gpd.read_file('shapefiles-master/india/city/india_cities.json')
# city_gdf = city_gdf[city_gdf["state"]=="Karnataka"]
city_gdf = city_gdf.to_crs(epsg=3857)

In [ ]:
for state in sorted(city_gdf["state"].unique()):
    print(state)
    for city in sorted(city_gdf[city_gdf["state"]==state]["city"].values):
        print("\t",city)

In [ ]:
city_1 = "Bangalore"
# city_2 = "Chikkamagaluru"
city_2 = "Delhi"

In [ ]:
city_info_dict = city_gdf[(city_gdf["city"]==city_1) | (city_gdf["city"]==city_2)]
                                                                                        # ,'lat','lon'])
city_info_dict["centroid"] = city_info_dict.apply(lambda x: Point(x["lon"],x["lat"]),axis=1)
city_info_dict.drop(columns=["cartodb_id","geometry","lat","lon"],inplace=True)
city_info_dict = city_info_dict.set_index("city").to_dict(orient="index")
pprint(city_info_dict,indent=4)

In [ ]:
gdf = gpd.read_file('india-master/taluk/india_taluk.geojson')
# gdf = gdf[gdf["NAME_1"]=="Karnataka"].reset_index(drop=True)
rename = {
    'ID_1': "St_ID",
    'NAME_1' : "State",
    'ID_2': "Dt_ID",
    'NAME_2' : "District",
    'ID_3' : "Tk_ID",
    'NAME_3' : "Taluk",
    'geometry':'geometry'
}
gdf.rename(columns=rename,inplace=True)
gdf = gdf[rename.values()]
display(gdf.head().T)
print(gdf.shape)
print(gdf.isna().sum())


In [ ]:
def AC_finder(gdf,point):

    return_value=False
    for index, row in gdf.iterrows():

        row_geometry = row['geometry']
        if row_geometry.contains(point):
            return_value = {
                    "Taluk" : row["Taluk"],
                    "Tk_ID" : row["Tk_ID"]}
            break

    return return_value


In [ ]:
from shapely.geometry import Point
starting_points = list(city_info_dict.keys())
for city in starting_points:
    return_value = AC_finder(gdf,city_info_dict[city]["centroid"])
    if return_value:
        city_info_dict[city] = {**city_info_dict[city], **return_value}
        city_info_dict[return_value["Tk_ID"]] = city_info_dict[city]
        del city_info_dict[city]
    else:
        print("City not found in any Taluks")
        sys.exit()
pprint(city_info_dict)


In [ ]:
projected_centroids = gdf.to_crs(epsg=3857).centroid
gdf["centroid"] = projected_centroids
gdf = gdf.to_crs(epsg=4326)
# gdf.plot()
display(gdf.head(2).T)
gdf_dict = gdf.to_dict(orient="index")
gdf_shape = gdf.shape[0]
# del gdf

In [ ]:
def dist_to_location(current_centroid, destination_centroid, random_multiplier=False):
    random_multiplier = 1 if random_multiplier else 0
    distance = current_centroid.distance(destination_centroid)
    return_dict = {
        "st_line_distance" : round(distance/1000),
        "rail_distance" : round((distance * ( 1 + random_multiplier * (random.uniform(0, .42))))/1000),
        "road_distance" : round((distance * ( 1 + random_multiplier * (random.uniform(.42, 1))))/1000),
                    }
    return return_dict


In [ ]:
adjacent_pairs_dict_karnataka = {}

for i in tqdm(gdf_dict):
    centroid_of_i = gdf_dict[i]["centroid"]
    i_dict = gdf_dict[i]
   
    i_dict["Adjacent_to"]={}
    i_dict["Destinations"]={
        city: dist_to_location(centroid_of_i, city_info_dict[city]["centroid"], random_multiplier=False) for city in city_info_dict
    }
    for j in range(i + 1, gdf_shape):
        j_dict = gdf_dict[j]
        try:
            if i_dict["geometry"].boundary.intersects(j_dict["geometry"].boundary):
                if i_dict["geometry"].intersection(j_dict["geometry"].boundary).length > 0:
                    i_dict["Adjacent_to"][j_dict["Tk_ID"]] =dist_to_location(centroid_of_i, j_dict["centroid"], random_multiplier=True)
        except AttributeError:
            pass
    adjacent_pairs_dict_karnataka[i] = i_dict

In [ ]:
node_maker_list = []
edge_maker_list = []
for index in adjacent_pairs_dict_karnataka:
    node_maker_list.append((adjacent_pairs_dict_karnataka[index]["Tk_ID"],adjacent_pairs_dict_karnataka[index]))
    for neighbor in adjacent_pairs_dict_karnataka[index]["Adjacent_to"]:
        
        edge_maker_list.append(
            (
                adjacent_pairs_dict_karnataka[index]["Tk_ID"],
                neighbor,
                adjacent_pairs_dict_karnataka[index]["Adjacent_to"][neighbor]
            )
        )
G = nx.Graph()
G.add_nodes_from(node_maker_list)
G.add_edges_from(edge_maker_list)

In [ ]:
pprint(G[adjacent_pairs_dict_karnataka[0]["Tk_ID"]])
pprint(G.nodes.data()[adjacent_pairs_dict_karnataka[0]["Tk_ID"]])

In [ ]:

import plotly.graph_objects as go

def plot_graph(graph_name,G):
    texts=[]
    for node in G.nodes:
        text = ""
        if len(G.nodes[node])>0:
            text += f"\n{G.nodes[node]['State']} - {G.nodes[node]['District']} - {G.nodes[node]['Taluk']}"
        texts.append(text)
        

    pos = nx.spring_layout(G)

    # Step 3: Extract node attributes for coloring
    node_attributes = nx.get_node_attributes(G, 'St_ID')  # Get the 'attribute' values
    node_colors = set(node_attributes.values())  # Convert attributes to a list

    min_color = min(node_colors)
    max_color = max(node_colors)
    normalized_colors = [(value - min_color) / (max_color - min_color) for value in node_colors]

    # Step 4: Create edge traces
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scatter(
        x=edge_x,
        y=edge_y,
        line=dict(width=0.5, color='gray'),
        hoverinfo='none',
        mode='lines'
    )

    # Step 5: Create node traces
    node_x = [pos[node][0] for node in G.nodes()]
    node_y = [pos[node][1] for node in G.nodes()]

    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers',
        marker=dict(
            size=20,
            color=normalized_colors,  # Use normalized colors for dynamic coloring
            colorscale='RdBu',  # Choose a color scale (e.g., Viridis, Plasma, etc.)
            showscale=False,  # Display the color scale legend
            # colorbar=dict(
            #     title="Node Attribute",
            #     thickness=15,
            #     xanchor="left",
            #     # titleside="right"
            # )
        ),
        # text=[f"Node {node}: Attribute {node_attributes[node]}" for node in G.nodes()],
        text = texts,
        # text=[G.nodes[node] for node in G.nodes()],
        hoverinfo="text"
    )

    # Step 6: Create the figure and plot it
    fig = go.Figure(data=[edge_trace, node_trace])
    fig.update_layout(
        title= graph_name,
        title_x=0.5,
        showlegend=False,
        margin=dict(l=40, r=40, t=40, b=40),
        xaxis=dict(showgrid=False, zeroline=False),
        yaxis=dict(showgrid=False, zeroline=False)
    )

    fig.show()

plot_graph("Indian talukas graph",G)


In [ ]:
pprint(G[adjacent_pairs_dict_karnataka[0]["Tk_ID"]])
pprint(G.nodes.data()[adjacent_pairs_dict_karnataka[0]["Tk_ID"]])

In [ ]:

def greedy_best_first_search(graph, start, goal, weight_type):
    # Priority queue: (heuristic, node, path)
    open_list = []
    heapq.heappush(open_list, (graph.nodes[start]['Destinations'][goal][weight_type], start, [start]))
    visited = set()
    
    nodes_generated = 0
    space_complexity = 0  # Maximum size of open_list and closed_list
    
    while open_list:
        h_val, current, path = heapq.heappop(open_list)

        if current in visited:
            continue
        visited.add(current)
        space_complexity = max(space_complexity, len(open_list) + len(visited))
        if current == goal:
            # Calculate total cost of the path for the selected weight type
            total_cost = 0
            for i in range(len(path) - 1):
                u, v = path[i], path[i + 1]
                total_cost += graph[u][v][weight_type]
            return path, total_cost, nodes_generated, space_complexity

        for neighbor in graph.neighbors(current):
            if neighbor not in visited:
                new_path = path + [neighbor]
                heapq.heappush(open_list, (graph[current][neighbor][weight_type], neighbor, new_path))
                nodes_generated += 1

    return None, float('inf')  # No path found


In [ ]:
GBFS_dict = {i:{} for i in ['st_line_distance','rail_distance', 'road_distance']}
for weight_type in GBFS_dict:
    start_node = city_info_dict[list(city_info_dict.keys())[0]]['Tk_ID']
    goal_node = city_info_dict[list(city_info_dict.keys())[1]]['Tk_ID']
    # print(f'\nGreedy Best First Search {weight_type}: {G.nodes[start_node]["Taluk"]} to {G.nodes[goal_node]["Taluk"]}\n')
    final_path_start_goal, final_path_goal_start = [] , []
    final_nodes_generated , final_space_complexity,final_exec_time = 0, 0, 0, 
    
    while_flag = True
    # exec_time = []
    while (while_flag):
        start_time = time.time()
        path_start_goal, total_distance , nodes_generated, space_complexity = greedy_best_first_search(G, start_node, goal_node, weight_type)
        end_time = time.time()
        
        final_exec_time+= end_time - start_time
        final_nodes_generated += nodes_generated
        final_space_complexity += space_complexity

        start_time = time.time()
        path_goal_start, total_distance , nodes_generated, space_complexity = greedy_best_first_search(G, goal_node, start_node, weight_type)
        end_time = time.time()

        final_exec_time+= end_time - start_time
        final_nodes_generated += nodes_generated
        final_space_complexity += space_complexity

        if path_start_goal == path_goal_start[::-1]:
            final_path_start_goal += path_start_goal
            final_path_goal_start += path_goal_start
            while_flag = False
        else:
            count = 0
            for i in range(min(len(path_start_goal),len(path_goal_start))):
                if path_start_goal[i] == path_goal_start[-i-1]:
                    count+=1
                else:
                    break
            
            final_path_start_goal += path_start_goal[:count]
            final_path_goal_start += path_goal_start[:count]        
            start_node = path_start_goal[count]
            goal_node = path_goal_start[count]
            
            new_destinations = [start_node,goal_node]
            new_destinations_dict = gdf[gdf["Tk_ID"].isin(new_destinations)][["Tk_ID","centroid"]].set_index("Tk_ID").to_dict(orient="index")
            
            for node in G.nodes:
                centroid_of_node = G.nodes[node]["centroid"]
                for destination in new_destinations:
                    G.nodes[node]['Destinations'][destination] = dist_to_location(centroid_of_node, new_destinations_dict[destination]["centroid"], random_multiplier=False)    
    
    i = 0
    GBFS_dict[weight_type] = { "A": [], "B": [],"metrics": {}}

    while True:
        you = final_path_start_goal[i]
        friend = final_path_goal_start[i]
        # print(f'Step {i}: you are at {G.nodes[you]["Taluk"]}, the friend is at {G.nodes[friend]["Taluk"]}')

        GBFS_dict[weight_type]["A"].append(G.nodes[you]["Taluk"])
        GBFS_dict[weight_type]["B"].append(G.nodes[friend]["Taluk"])

        if (final_path_start_goal[i] == final_path_goal_start[i]):
            break
        elif (final_path_start_goal[i] == final_path_goal_start[i+1]):
            # print(f'Step {i+1}: You stay at {G.nodes[you]["Taluk"]}, the friend comes to {G.nodes[final_path_goal_start[i+1]]["Taluk"]}')
            GBFS_dict[weight_type]["B"].append(G.nodes[final_path_goal_start[i+1]]["Taluk"])
            break
        i+=1
    GBFS_dict[weight_type]["metrics"]["final_exec_time"] = final_exec_time
    GBFS_dict[weight_type]["metrics"]["final_nodes_generated"] = final_nodes_generated
    GBFS_dict[weight_type]["metrics"]["final_space_complexity"] = final_space_complexity


In [ ]:
# A* search function with multiple edge weights
def a_star_search(graph, start, goal, weight_type):
    open_list = []  # Priority queue for nodes to explore
    # heapq.heappush(open_list, (graph.nodes[start]['heuristic'], 0, start, [start]))  # (f, g, node, path)
    heapq.heappush(open_list, (graph.nodes[start]['Destinations'][goal][weight_type], 0, start, [start]))  # (f, g, node, path)
    closed_list = set()  # Set of explored nodes

    nodes_generated = 0
    space_complexity = 0  # Maximum size of open_list and closed_list
    
    while open_list:
        f, g, current, path = heapq.heappop(open_list)
        if current in closed_list:
            continue

        closed_list.add(current)
        space_complexity = max(space_complexity, len(open_list) + len(closed_list))

        if current == goal:
            return path, g , nodes_generated, space_complexity
            # Return the path total distance, nodes_generated and space_complexity

        for neighbor in graph.neighbors(current):
            if neighbor not in closed_list:

                g_new = g + graph[current][neighbor][weight_type]
                f_new = g_new + graph.nodes[neighbor]['Destinations'][goal][weight_type]
                heapq.heappush(open_list, (f_new, g_new, neighbor, path + [neighbor]))
                nodes_generated += 1

    return None, float('inf')  # Return None if no path is found

In [ ]:
start_node = city_info_dict[list(city_info_dict.keys())[0]]['Tk_ID']
goal_node = city_info_dict[list(city_info_dict.keys())[1]]['Tk_ID']

A_star_dict = {i:{} for i in ['st_line_distance','rail_distance', 'road_distance']}
for weight_type in A_star_dict:
    # print(f'\nA* Search {weight_type}: {G.nodes[start_node]["Taluk"]} to {G.nodes[goal_node]["Taluk"]}\n')
    final_path_start_goal, final_path_goal_start = [] , []
    final_nodes_generated , final_space_complexity,final_exec_time = 0, 0, 0  

    while_flag = True
    
    while (while_flag):
        start_time = time.time()
        path_start_goal, total_distance , nodes_generated_start_goal, space_complexity_start_goal = a_star_search(G, start_node, goal_node, weight_type)
        end_time = time.time()
        
        final_exec_time+= end_time - start_time
        final_nodes_generated += nodes_generated
        final_space_complexity += space_complexity

        start_time = time.time()
        path_goal_start, total_distance , nodes_generated_goal_start, space_complexity_goal_start = a_star_search(G, goal_node, start_node, weight_type)
        end_time = time.time()
        
        final_exec_time+= end_time - start_time
        final_nodes_generated += nodes_generated
        final_space_complexity += space_complexity
        
        if path_start_goal == path_goal_start[::-1]:
            final_path_start_goal = path_start_goal
            final_path_goal_start = path_goal_start
            while_flag = False
        else:
            count = 0
            for i in range(min(len(path_start_goal),len(path_goal_start))):
                if path_start_goal[i] == path_goal_start[-i-1]:
                    count+=1
                else:
                    break
            
            final_path_start_goal += path_start_goal[:count]
            final_path_goal_start += path_goal_start[:count]


            
            start_node = path_start_goal[count]
            goal_node = path_goal_start[count]
         
            new_destinations = [start_node,goal_node]
            new_destinations_dict = gdf[gdf["Tk_ID"].isin(new_destinations)][["Tk_ID","centroid"]].set_index("Tk_ID").to_dict(orient="index")

            pprint(new_destinations_dict)
            
            for node in G.nodes:
                centroid_of_node = G.nodes[node]["centroid"]
                for destination in new_destinations:
                    G.nodes[node]['Destinations'][destination] = dist_to_location(centroid_of_node, new_destinations_dict[destination]["centroid"], random_multiplier=False)    
    
    i = 0
    A_star_dict[weight_type] = { "A": [], "B": [],"metrics": {}}

    while True:
        you = final_path_start_goal[i]
        friend = final_path_goal_start[i]
        # print(f'Step {i}: you are at {G.nodes[you]["Taluk"]}, the friend is at {G.nodes[friend]["Taluk"]}')

        A_star_dict[weight_type]["A"].append(G.nodes[you]["Taluk"])
        A_star_dict[weight_type]["B"].append(G.nodes[friend]["Taluk"])

        if (final_path_start_goal[i] == final_path_goal_start[i]):
            break
        elif (final_path_start_goal[i] == final_path_goal_start[i+1]):
            # print(f'Step {i+1}: You stay at {G.nodes[you]["Taluk"]}, the friend comes to {G.nodes[final_path_goal_start[i+1]]["Taluk"]}')
            A_star_dict[weight_type]["B"].append(G.nodes[final_path_goal_start[i+1]]["Taluk"])
            break
        i+=1
    A_star_dict[weight_type]["metrics"]["final_exec_time"] = final_exec_time
    A_star_dict[weight_type]["metrics"]["final_nodes_generated"] = final_nodes_generated
    A_star_dict[weight_type]["metrics"]["final_space_complexity"] = final_space_complexity


In [ ]:
def plot_path(search_algo,search_dict):
    for weight_type in search_dict:
        # Create a directed graph
        weight_type_G = nx.DiGraph()
        edges = []
        
        for path in ["A","B"]:
            for i in range(len(search_dict[weight_type][path])-1):
                edge = (search_dict[weight_type][path][i], search_dict[weight_type][path][i+1],f"{path}'s turn {i+1}")
                edges.append(edge)
                # edge_labels[(search_dict[weight_type][path][i], search_dict[weight_type][path][i+1])] = f"{path}'s turn {i+1} "
        
        weight_type_G.add_edges_from([(u, v, {'label': label}) for u, v, label in edges])
        pos = nx.spring_layout(weight_type_G)
        edge_traces = []
        
        for edge in weight_type_G.edges(data=True):
            # print(edge)
            # sys.exit()
            x0, y0 = pos[edge[0]]
            x1, y1 = pos[edge[1]]
            colour = 'green' if edge[2]['label'][0] == "A" else 'red'

            edge_trace = go.Scatter(
                x=[x0, (x0+x1)/2, x1, None], 
                y=[y0, (y0+y1)/2, y1, None],
                line=dict(width=1.5, color= colour #'#888'
                        ),
                hoverinfo='text',
                text=['', edge[2]['label'], '', None],  # Only middle point has hover text
                mode='lines+markers',
                marker=dict(size=[0, 6, 0, 0], color= colour #'#888'
                            )
            )
            edge_traces.append(edge_trace)

        # Create node trace
        node_x = [pos[node][0] for node in weight_type_G.nodes()]
        node_y = [pos[node][1] for node in weight_type_G.nodes()]

        node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        text=list(weight_type_G.nodes()),
        textposition="middle center",
        textfont=dict(color='blue'),  # Set text color to blue
        hoverinfo='text',
        marker=dict(
            color='gray',  # Set node color to gray
            size=40,
            line=dict(width=2)
        )
    )

        
        # Create arrow annotations
        annotations = []
        for edge in weight_type_G.edges(data=True):
            # print(edge)
            # sys.exit()
            x0, y0 = pos[edge[0]]
            x1, y1 = pos[edge[1]]
            colour = 'green' if edge[2]['label'][0] == "A" else 'red'
            annotations.append(
                dict(
                    ax=x0, ay=y0,
                    x=x1, y=y1,
                    xref='x', yref='y',
                    axref='x', ayref='y',
                    showarrow=True,
                    arrowhead=2,
                    arrowsize=2,
                    arrowwidth=1,
                    arrowcolor= colour
                    #'#888'
                )
            )

        # Create figure
        fig = go.Figure(
            data=edge_traces + [node_trace],
            layout=go.Layout(
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=annotations,
                xaxis=dict(showgrid=False, zeroline=False),
                yaxis=dict(showgrid=False, zeroline=False),
                plot_bgcolor='white'
            )
        )

        # Add interaction features
        fig.update_layout(
            clickmode='event+select',
            dragmode='pan',
            hoverlabel=dict(
                bgcolor="white",
                font_size=16,
                font_family="Rockwell"
            )
        )

        # Add title
        fig.update_layout(title_text= f'{search_algo} Search {weight_type}:\tA from {G.nodes[start_node]["Taluk"]} & B from {G.nodes[goal_node]["Taluk"]}',
                        #   f'{search_algo} {weight_type}',
                          title_x=0.5,
                          xaxis=dict(
            showgrid=False,
            zeroline=False,
            showticklabels=False,
            visible=False  # Hide axis line
        ),
        yaxis=dict(
            showgrid=False,
            zeroline=False,
            showticklabels=False,
            visible=False  # Hide axis line
        )
        )
        # Show interactive plot
        fig.show()
        # pprint(search_dict[weight_type],indent=4,depth=2)
        for path in ["A","B"]:
            print(f"\n{path}'s path in {len(search_dict[weight_type][path])} turns:\n{" -> ".join(search_dict[weight_type][path])}")
        for metric in search_dict[weight_type]["metrics"]:
            print(f"{metric}: {search_dict[weight_type]['metrics'][metric]}")
        # sys.exit()

In [ ]:
plot_path(search_algo = "GBFS",search_dict = GBFS_dict)
plot_path(search_algo = "A*",search_dict = A_star_dict)

In [ ]:
sys.exit()